<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


## Practice Project - Create a machine learning pipeline for a regression project


Estimated time needed: **90** minutes


## Scenario


You are a data engineer at a data analytics consulting company. Your company prides itself in being able to efficiently handle huge datasets. Data scientists in your office need to work with different algorithms and data in different formats. While they are good at Machine Learning, they count on you to be able do ETL jobs and build ML pipelines.



## Objectives

In this 4 part assignment you will:

- Part 1 ETL
  - Load a csv dataset
  - Remove duplicates if any
  - Drop rows with null values if any
  - Make transformations
  - Store the cleaned data in parquet format
- Part 2 Machine Learning Pipeline creation
  - Create a machine learning pipeline for prediction
- Part 3 Model evaluation
  - Evaluate the model using metrics
  - Print the intercept and the coefficients
- Part 4 Model Persistance
  - Cave the model for future production use
  - Load and verify the stored model


## Datasets

In this lab you will be using dataset(s):

 - Modified version of car mileage dataset. Original dataset available at https://archive.ics.uci.edu/ml/datasets/auto+mpg 
 


----


## Setup


For this lab, we will be using the following libraries:

*   [`PySpark`](https://spark.apache.org/docs/latest/api/python/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01) for connecting to the Spark Cluster


### Installing Required Libraries

Spark Cluster is pre-installed in the Skills Network Labs environment. However, you need libraries like pyspark and findspark to
 connect to this cluster.


The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

## Part 1 - ETL


### Task 1 - Import required libraries


In [3]:
# Import libraries
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import when, count, col
import os
from pyspark.ml.evaluation import RegressionEvaluator

<details>
    <summary>Click here for a Hint</summary>
    
Import all the required libraries
    
</details>


<details>
    <summary>Click here for Solution</summary>

```python
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler

```

</details>


### Task 2 - Create a spark session


In [4]:
# Create SparkSession
spark = SparkSession.builder.appName('Practice Project').getOrCreate()

24/02/12 03:52:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/12 03:52:32 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<details>
    <summary>Click here for a Hint</summary>
    
Use the SparkSession.builder

</details>


<details>
    <summary>Click here for Solution</summary>

```python
spark = SparkSession.builder.appName("Practice Project").getOrCreate()
```

</details>


### Task 3 - Load the csv file into a dataframe


Download the data file


In [5]:
# Download the file
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/mpg-raw.csv

--2024-02-12 03:52:35--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/mpg-raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14354 (14K) [text/csv]
Saving to: ‘mpg-raw.csv.2’

mpg-raw.csv.2       100%[===================>]  14.02K  --.-KB/s    in 0s      

2024-02-12 03:52:35 (38.3 MB/s) - ‘mpg-raw.csv.2’ saved [14354/14354]



Load the dataset into the spark dataframe


In [6]:
# Load dataset
df = spark.read.csv('mpg-raw.csv', header=True, inferSchema=True)

<details>
    <summary>Click here for a Hint</summary>
    
Use  spark.read.csv

</details>


<details>
    <summary>Click here for Solution</summary>

```python
df = spark.read.csv("mpg-raw.csv", header=True, inferSchema=True)

```

</details>


### Task 4 - Print top 5 rows of the dataset


In [7]:
# Show the first five rows
df.show(5)

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|46.6|        4|       86.0|        65|  2110|      17.9|  80|Japanese|
|44.6|        4|       91.0|        67|  1850|      13.8|  80|Japanese|
|44.3|        4|       90.0|        48|  2085|      21.7|  80|European|
|44.0|        4|       97.0|        52|  2130|      24.6|  82|European|
|43.4|        4|       90.0|        48|  2335|      23.7|  80|European|
+----+---------+-----------+----------+------+----------+----+--------+
only showing top 5 rows



### Task 5 - Print the number of cars in each Origin


In [8]:
# Print the number of cars in each origin
df.groupBy('Origin').count().orderBy('count').show()

+--------+-----+
|  Origin|count|
+--------+-----+
|    null|    1|
|European|   70|
|Japanese|   88|
|American|  247|
+--------+-----+



<details>
    <summary>Click here for a Hint</summary>
    
Use  df.groupBy

</details>


<details>
    <summary>Click here for Solution</summary>

```python
df.groupBy('Origin').count().orderBy('count').show()
```

</details>


### Task 6 - Print the total number of rows in the dataset


In [9]:
# Print the total number of rows 
rowcount1 = df.count()
print(rowcount1)

406


### Task 7 - Drop all the duplicate rows from the dataset


In [10]:
# Check duplicates
duplicates_count = df.exceptAll(df.dropDuplicates()).count()
print(duplicates_count)

[Stage 9:====================================================>  (191 + 9) / 200]

14


In [11]:
# Drop all the duplicate rows from the dataset
df = df.dropDuplicates()

<details>
    <summary>Click here for a Hint</summary>
    
Use  df.dropDuplicates

</details>


<details>
    <summary>Click here for Solution</summary>

```python
df = df.dropDuplicates()
```

</details>


### Task 8 - Print the total number of rows in the dataset


In [12]:
# Check the number of rows after dropping duplicates
rowcount2 = df.count()
print(rowcount2)

[Stage 12:=================================================>    (183 + 8) / 200]

392


### Task 9 - Drop all the rows that contain null values from the dataset


In [13]:
# Check for missing values
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

[Stage 15:===================================================>  (189 + 8) / 200]

+---+---------+-----------+----------+------+----------+----+------+
|MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|Origin|
+---+---------+-----------+----------+------+----------+----+------+
|  1|        1|          1|         1|     1|         1|   0|     1|
+---+---------+-----------+----------+------+----------+----+------+



In [14]:
# Drop the missing values
df = df.dropna()

<details>
    <summary>Click here for a Hint</summary>
    
Use df.dropna

</details>


<details>
    <summary>Click here for Solution</summary>

```python
df=df.dropna()
```

</details>


### Task 10 - Print the total number of rows in the dataset


In [15]:
# Check the number of rows after dropping missing values
rowcount3 = df.count()
print(rowcount3)

[Stage 18:==================================================>   (188 + 8) / 200]

385


### Task 11 - Rename the column "Engine Disp" to "Engine_Disp"


In [16]:
# Rename the column Engine Disp" to "Engine_Disp"
df = df.withColumnRenamed('Engine Disp', 'Engine_Disp')

<details>
    <summary>Click here for a Hint</summary>
    
Use df.withColumnRenamed

</details>


<details>
    <summary>Click here for Solution</summary>

```python
df = df.withColumnRenamed("Engine Disp","Engine_Disp")
```

</details>


### Task 12 - Save the dataframe in parquet format, name the file as "mpg-cleaned.parquet"


In [17]:
# Reduce the number of partitions in the dataframe to one
df = df.repartition(1)

# Save the dataframe in parquet format
df.write.mode('overwrite').parquet('mpg-cleaned.parquet')

<details>
    <summary>Click here for a Hint</summary>
    
Use df.write.parquet

</details>


<details>
    <summary>Click here for Solution</summary>

```python
df.write.parquet("mpg-cleaned.parquet")
```

</details>


#### Part 1 - Evaluation



Run the code cell below.<br>
If the code throws up any errors, go back and review the code you have written.


In [18]:
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("Renamed column name = ", df.columns[2])

print("mpg-cleaned.parquet exists :", os.path.isdir("mpg-cleaned.parquet"))

Part 1 - Evaluation
Total rows =  406
Total rows after dropping duplicate rows =  392
Total rows after dropping duplicate rows and rows with null values =  385
Renamed column name =  Engine_Disp
mpg-cleaned.parquet exists : True


## Part - 2 Machine Learning Pipeline creation


### Task 1 - Load data from "mpg-cleaned.parquet" into a dataframe


In [19]:
# Load data from "mpg-cleaned.parquet" into a dataframe
df = spark.read.parquet('mpg-cleaned.parquet')
rowcount4 = df.count()
print(rowcount4)

385


In [20]:
# Show the top 5 rows
df.show(5)

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine_Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|24.0|        4|      134.0|        96|  2702|      13.5|  75|Japanese|
|18.0|        6|      250.0|        88|  3139|      14.5|  71|American|
|29.0|        4|       68.0|        49|  1867|      19.5|  73|European|
|22.4|        6|      231.0|       110|  3415|      15.8|  81|American|
|20.5|        6|      231.0|       105|  3425|      16.9|  77|American|
+----+---------+-----------+----------+------+----------+----+--------+
only showing top 5 rows



In [21]:
# Print the schema of the dataframe
df.printSchema()

root
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Engine_Disp: double (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Accelerate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Origin: string (nullable = true)



### Task 2 - Define the StringIndexer pipeline stage


In [22]:
# Stage - 1 Using StringIndexer convert the string column 'Origin' into 'OriginIndex'
indexer = StringIndexer(inputCol='Origin', outputCol='OriginIndex')

<details>
    <summary>Click here for a Hint</summary>
    
Use StringIndexer class

</details>


<details>
    <summary>Click here for Solution</summary>

```python
indexer = StringIndexer(inputCol="Origin", outputCol="OriginIndex")
```

</details>


### Task 3 - Define the VectorAssembler pipeline stage


In [23]:
# Stage 2 - Assemble the input columns 'Cylinders','Engine_Disp','Horsepower','Weight','Accelerate','Year' into a single column 'features'
assembler = VectorAssembler(inputCols=['Cylinders','Engine_Disp','Horsepower','Weight','Accelerate','Year'], outputCol='features')

<details>
    <summary>Click here for a Hint</summary>
    
Use the VectorAssembler class

</details>


<details>
    <summary>Click here for Solution</summary>

```python
assembler = VectorAssembler(inputCols=['Cylinders','Engine_Disp','Horsepower','Weight','Accelerate','Year'], outputCol="features")
```

</details>


### Task 4 - Define the StandardScaler pipeline stage


In [24]:
# Stage 3 - scale the "features" using standard scaler and store in "scaledFeatures" column
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

<details>
    <summary>Click here for a Hint</summary>
    
Use the StandardScaler class

</details>


<details>
    <summary>Click here for Solution</summary>

```python
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

```

</details>


### Task 5 - Define the StandardScaler pipeline stage


In [25]:
# Stage 4 - Create a LinearRegression stage to predict "MPG"
lr = LinearRegression(featuresCol='features', labelCol='MPG')

<details>
    <summary>Click here for a Hint</summary>
    
Use the LinearRegression class

</details>


<details>
    <summary>Click here for Solution</summary>

```python
lr = LinearRegression(featuresCol="features", labelCol="MPG")
```

</details>


### Task 6 - Build the pipeline


In [26]:
# Build a pipeline using the above four stages
pipeline = Pipeline(stages=[indexer, assembler, scaler, lr])

<details>
    <summary>Click here for a Hint</summary>
    
Use the Pipeline class

</details>


<details>
    <summary>Click here for Solution</summary>

```python
pipeline = Pipeline(stages=[indexer,assembler, scaler, lr])
```

</details>


### Task 7 - Split the data


In [27]:
# Split the data into training and testing sets with 70:30 split. Use 42 as seed
(trainingData, testingData) = df.randomSplit([0.7, 0.3], seed=42)

<details>
    <summary>Click here for a Hint</summary>
    
Use the randomSplit method

</details>


<details>
    <summary>Click here for Solution</summary>

```python
(trainingData, testingData) = df.randomSplit([0.7, 0.3], seed=42)
```

</details>


### Task 8 - Fit the pipeline


In [28]:
# Fit the pipeline using the training data
pipelineModel = pipeline.fit(trainingData)

24/02/12 03:53:35 WARN util.Instrumentation: [910c7cc2] regParam is zero, which might cause numerical instability and overfitting.
24/02/12 03:53:35 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/02/12 03:53:35 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/02/12 03:53:35 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/02/12 03:53:35 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


<details>
    <summary>Click here for a Hint</summary>
    
Use the pipeline.fit method

</details>


<details>
    <summary>Click here for Solution</summary>

```python
pipelineModel = pipeline.fit(trainingData)
```

</details>


#### Part 2 - Evaluation



Run the code cell below.<br>
If the code throws up any errors, go back and review the code you have written.


In [29]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  385
Pipeline Stage 1 =  StringIndexer
Pipeline Stage 2 =  VectorAssembler
Pipeline Stage 3 =  StandardScaler
Label column =  MPG


## Part 3 - Model Evaluation


### Task 1 - Predict using the model


In [30]:
# Make predictions on testing data
predictions = pipelineModel.transform(testingData)

<details>
    <summary>Click here for a Hint</summary>
    
Use the transform method of the model

</details>


<details>
    <summary>Click here for Solution</summary>

```python
predictions = pipelineModel.transform(testingData)
```

</details>


### Task 2 - Print the MSE


In [31]:
# Calculate the MSE
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='MPG', metricName='mse')
mse = evaluator.evaluate(predictions)
print('MSE:', mse)

MSE: 17.16352626871954


<details>
    <summary>Click here for a Hint</summary>
    
Use the RegressionEvaluator

</details>


<details>
    <summary>Click here for Solution</summary>

```python
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="MPG", metricName="mse")
mse = evaluator.evaluate(predictions)
print(mse)

```

</details>


### Task 3 - Print the MAE


In [32]:
# Calculate the MAE
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='MPG', metricName='mae')
mae = evaluator.evaluate(predictions)
print('MAE:', mae)

MAE: 3.1789138927145015


<details>
    <summary>Click here for a Hint</summary>
    
Use the RegressionEvaluator

</details>


<details>
    <summary>Click here for Solution</summary>

```python
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="MPG", metricName="mae")
mae = evaluator.evaluate(predictions)
print(mae)

```

</details>


### Task 4 - Print the R-Squared(R2)


In [33]:
# Calculate the R-squared score
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='MPG', metricName='r2')
r2 = evaluator.evaluate(predictions)
print('R2:', r2)

R2: 0.7323919302279742


<details>
    <summary>Click here for a Hint</summary>
    
Use the RegressionEvaluator

</details>


<details>
    <summary>Click here for Solution</summary>

```python
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="MPG", metricName="r2")
r2 = evaluator.evaluate(predictions)
print(r2)
```

</details>


#### Part 3 - Evaluation



Run the code cell below.<br>
If the code throws up any errors, go back and review the code you have written.


In [34]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = pipelineModel.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))

Part 3 - Evaluation
Mean Squared Error =  17.16
Mean Absolute Error =  3.18
R Squared =  0.73
Intercept =  -11.74


## Part 4 - Model persistance


### Task 1 - Save the model to the path "Practice_Project"


In [35]:
# Save the pipeline model
pipelineModel.write().overwrite().save('practice_project')

<details>
    <summary>Click here for a Hint</summary>
    
Use the model.write().save method

</details>


<details>
    <summary>Click here for Solution</summary>

```python
pipelineModel.write().save("Practice_Project")
```

</details>


### Task 2 - Load the model from the path "Practice_Project"


In [36]:
# Load the pipeline model
loaded_model = PipelineModel.load('practice_project')

<details>
    <summary>Click here for a Hint</summary>
    
Use the load method of the model

</details>


<details>
    <summary>Click here for Solution</summary>

```python
loadedPipelineModel = PipelineModel.load("Practice_Project")
```

</details>


### Task 3 - Make predictions using the loaded model on the test data


In [37]:
# Use the loaded pipeline model for predictions
loaded_predictions = loaded_model.transform(testingData)

<details>
    <summary>Click here for a Hint</summary>
    
Use the transform method

</details>


<details>
    <summary>Click here for Solution</summary>

```python
predictions = loadedPipelineModel.transform(testingData)
```

</details>


### Task 4 - Show the predictions


In [38]:
# Show the predictions from the loaded model
loaded_predictions.select('MPG', 'prediction').show()

+----+------------------+
| MPG|        prediction|
+----+------------------+
|10.0| 9.717098109895478|
|10.0| 8.470953440738334|
|11.0| 9.767331545650293|
|13.0| 21.73313441531174|
|13.0|12.610630120433388|
|13.0| 9.878726564012467|
|13.0|11.487015801415524|
|13.0| 8.046959634888928|
|14.0| 10.08968878222387|
|14.0| 16.58966164732882|
|14.0| 20.97653635178814|
|14.0|10.687717376033103|
|15.0|21.702580500435005|
|15.0|19.689505748749095|
|15.0|18.791796125953226|
|15.0|12.720613041387747|
|15.0|16.106906328481617|
|15.0|10.834316938930069|
|16.0|16.270144700047833|
|16.0|12.037361781744458|
+----+------------------+
only showing top 20 rows



<details>
    <summary>Click here for a Hint</summary>
    
Use the the select method of the dataframe

</details>


<details>
    <summary>Click here for Solution</summary>

```python
predictions.select("MPG","prediction").show()
```

</details>


#### Part 4 - Evaluation



Run the code cell below.<br>
If the code throws up any errors, go back and review the code you have written.


In [39]:
print("Part 4 - Evaluation")

loadedmodel = loaded_model.stages[-1]
totalstages = len(loaded_model.stages)
inputcolumns = loaded_model.stages[1].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i,j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

Part 4 - Evaluation
Number of stages in the pipeline =  4
Coefficient for Cylinders is -0.1998
Coefficient for Engine_Disp is 0.0099
Coefficient for Horsepower is 0.009
Coefficient for Weight is -0.0077
Coefficient for Accelerate is 0.068
Coefficient for Year is 0.7262


### Task 5 - Stop Spark Session


In [40]:
spark.stop()

Congratulations!! you have successfully finished the practice project.


## Authors


[Ramesh Sannareddy](https://www.linkedin.com/in/rsannareddy/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMBD0231ENSkillsNetwork866-2023-01-01)


### Other Contributors


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-05-26|0.1|Ramesh Sannareddy|Initial Version Created|


Copyright © 2023 IBM Corporation. All rights reserved.
